In [2]:
from qwen2.modeling_qwen2 import Qwen2ForCausalLM
from qwen2.configuration_qwen2 import Qwen2Config
from transformers import AutoTokenizer
import torch
from copy import deepcopy
from tqdm import tqdm

In [5]:
model = Qwen2ForCausalLM.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-7B", device_map='cuda:3', attn_implementation="sdpa", partial_rotary_factor=4)
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-7B")
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.68s/it]
Some weights of Qwen2ForCausalLM were not initialized from the model checkpoint at /root/mfx/huggingface/Qwen/Qwen2.5-7B and are newly initialized: ['model.layers.0.self_attn.k_up_proj.weight', 'model.layers.0.self_attn.v_up_proj.weight', 'model.layers.1.self_attn.k_up_proj.weight', 'model.layers.1.self_attn.v_up_proj.weight', 'model.layers.10.self_attn.k_up_proj.weight', 'model.layers.10.self_attn.v_up_proj.weight', 'model.layers.11.self_attn.k_up_proj.weight', 'model.layers.11.self_attn.v_up_proj.weight', 'model.layers.12.self_attn.k_up_proj.weight', 'model.layers.12.self_attn.v_up_proj.weight', 'model.layers.13.self_attn.k_up_proj.weight', 'model.layers.13.self_attn.v_up_proj.weight', 'model.layers.14.self_attn.k_up_proj.weight', 'model.layers.14.self_attn.v_up_proj.weight', 'model.layers.15.self_attn.k_up_proj.weight', 'model.layers.15.self_attn.v_up_proj.weight', 'model.layers.16.self_attn.k_up_proj.weight', 

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaMLAttention(
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (k_up_proj): Linear(in_features=512, out_features=3584, bias=False)
          (v_up_proj): Linear(in_features=512, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
      

In [6]:
hidden_size = model.config.hidden_size
n_heads = model.config.num_attention_heads
kv_heads = model.config.num_key_value_heads
head_dim = model.config.hidden_size//model.config.num_attention_heads
latent_dim = kv_heads * head_dim
kv_groups = model.config.num_attention_heads // model.config.num_key_value_heads
model.config.partial_rotary_factor

4

In [7]:
# Insert identity matrices
for name,module in model.named_modules():
    if 'k_up_proj' in name or "v_up_proj" in name:
        weight = torch.stack([torch.eye(latent_dim).reshape(kv_heads, head_dim, latent_dim)]*kv_groups,dim=1).reshape(hidden_size, latent_dim).contiguous().to(module.weight.data.device,module.weight.data.dtype)
        if 'k_up_proj' in name:
            weight = weight.view(hidden_size, kv_heads, head_dim).transpose(1,2).contiguous().view(hidden_size, latent_dim)
        module.weight.data=weight
    elif 'k_proj' in name:
        module.weight.data = module.weight.data.view(kv_heads, head_dim, hidden_size).transpose(0,1).contiguous().view(latent_dim, hidden_size)
        module.bias.data = module.bias.data.view(kv_heads, head_dim).transpose(0,1).contiguous().view(latent_dim)

In [8]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:3"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它会错过展示自己美丽歌声的机会，也会让朋友们失望。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你会发现飞翔其实并不难。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它深吸一口气，闭上眼睛，然后张开翅膀，飞向天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力调整姿态，逐渐掌握了飞行的技巧。当它再次回到地面时，所有的动物都为它鼓掌欢呼。蓝羽不仅赢得了比赛，还证明了自己不仅有美丽的歌声，也有惊人的勇气。

从那以后，蓝羽成为了森林中的英雄，它的故事激励着每一个害怕尝试新事物的生物。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起探索森林的每一个角落，享受着飞行带来的快乐。

这个故事告诉我们，无论我们多么害怕，只要我们勇敢地迈出第一步，就能发现自己的潜力和勇气。<|endoftext|>


In [9]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Orthogonal q_proj and k_up_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(q_k_up, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->hdD',torch.sqrt(S),V) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            module.q_proj.bias.data = S_sqrtV[:,:,-1].reshape(-1).contiguous()
            S_sqrtV = S_sqrtV[:,:,:-1]
        module.k_up_proj.weight.data = US_sqrt.reshape(n_heads*head_dim, latent_dim).contiguous()
        module.q_proj.weight.data = S_sqrtV.reshape(n_heads*head_dim, hidden_size).contiguous()

        # Orthogonal o_proj and v_up_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim)
        v_up_o = torch.einsum("hdc,Dhd->hcD",v_up_weight, o_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        U,S,V = torch.svd_lowrank(v_up_o, head_dim, niter=16) # U(n_heads, latent_dim, head_dim), S(n_heads, head_dim), V(n_heads, hidden_size, head_dim)
        US_sqrt = torch.einsum('hLd,hd->hdL',U,torch.sqrt(S)) # (n_heads, head_dim, latent_dim)
        S_sqrtV = torch.einsum('hd,hDd->Dhd',torch.sqrt(S),V) # (hidden_size, n_heads, head_dim)
        module.v_up_proj.weight.data = US_sqrt.reshape(hidden_size, latent_dim).contiguous()
        module.o_proj.weight.data = S_sqrtV.reshape(hidden_size, n_heads*head_dim).contiguous()

In [10]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:3"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它会错过展示自己美丽歌声的机会，也会让朋友们失望。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你会发现飞翔其实并不难。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它深吸一口气，闭上眼睛，然后张开翅膀，飞向天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力调整姿态，逐渐掌握了飞行的技巧。当它再次回到地面时，所有的动物都为它鼓掌欢呼。蓝羽不仅赢得了比赛，还证明了自己不仅有美丽的歌声，也有惊人的勇气。

从那以后，蓝羽成为了森林中的英雄，它的故事激励着每一个害怕尝试新事物的生物。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起探索森林的每一个角落，享受着飞行带来的快乐。

这个故事告诉我们，无论我们多么害怕，只要我们勇敢地迈出第一步，就能发现自己的潜力和勇气。<|endoftext|>


In [11]:
for name,module in model.named_modules():
    if name.endswith("self_attn"):
        # Absorb k_up_proj into q_proj
        k_up_weight = deepcopy(module.k_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        q_weight = deepcopy(module.q_proj.weight.data).reshape(n_heads, head_dim, hidden_size) # (n_heads, head_dim, hidden_size)
        if module.q_proj.bias is not None:
            q_weight = torch.cat([q_weight,deepcopy(module.q_proj.bias.data).reshape(n_heads, head_dim, 1)],dim=-1)
        q_k_up = torch.einsum("hdc,hdD->hcD",k_up_weight, q_weight) # (n_heads, latent_dim, hidden_size), rank<=head_dim
        q_proj = torch.nn.Linear(hidden_size, n_heads*latent_dim, bias=(module.q_proj.bias is not None))
        q_proj = q_proj.to(device=module.q_proj.weight.device, dtype=module.q_proj.weight.dtype)
        if module.q_proj.bias is not None:
            q_proj.bias.data = q_k_up[:,:,-1].reshape(-1).contiguous()
            q_k_up = q_k_up[:,:,:-1]
        q_proj.weight.data = q_k_up.reshape(n_heads*latent_dim, hidden_size).contiguous()
        setattr(module, "q_proj", q_proj)
        delattr(module, "k_up_proj")
        # Absorb v_up_proj into o_proj
        v_up_weight = deepcopy(module.v_up_proj.weight.data).reshape(n_heads, head_dim, latent_dim) # (n_heads, head_dim, latent_dim)
        o_weight = deepcopy(module.o_proj.weight.data).reshape(hidden_size, n_heads, head_dim) # (n_heads, head_dim, hidden_size)
        v_up_o = torch.einsum("hdc,Dhd->Dhc",v_up_weight, o_weight) # (hidden_size, n_heads, latent_dim), rank<=head_dim
        o_proj = torch.nn.Linear(n_heads*latent_dim, hidden_size, bias=(module.o_proj.bias is not None))
        o_proj = o_proj.to(device=module.o_proj.weight.device, dtype=module.o_proj.weight.dtype)
        o_proj.weight.data = v_up_o.reshape(hidden_size, n_heads*latent_dim).contiguous()
        if module.o_proj.bias is not None:
            o_proj.bias.data = module.o_proj.bias
        setattr(module, "o_proj", o_proj)
        delattr(module, "v_up_proj")
        module.absorb = True

In [12]:
output = model.generate(**tokenizer("给我讲一个故事吧",return_tensors="pt").to("cuda:3"), max_new_tokens=500, do_sample=False)
print(tokenizer.batch_decode(output)[0])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


给我讲一个故事吧。
当然可以，这是一个关于勇气和友谊的故事：

从前，在一个遥远的森林里，住着一只名叫蓝羽的小鸟。蓝羽有着美丽的蓝色羽毛和甜美的歌声，但它有一个秘密——它害怕飞翔。每当其他鸟儿在天空中自由翱翔时，蓝羽总是躲在树枝上，不敢加入它们。

一天，森林里的动物们决定举办一场飞行比赛，以庆祝春天的到来。所有的鸟儿都兴奋地准备着，只有蓝羽感到非常焦虑。它知道如果不去参加比赛，它会错过展示自己美丽歌声的机会，也会让朋友们失望。

就在这时，一只名叫风灵的老鹰飞到了蓝羽的树上。风灵是森林中最勇敢的鸟儿之一，它看到了蓝羽的不安。风灵对蓝羽说：“蓝羽，我知道你害怕飞翔，但请相信我，只要你勇敢地迈出第一步，你会发现飞翔其实并不难。”

受到风灵的鼓励，蓝羽鼓起勇气，决定参加比赛。在比赛的那天，所有的动物都聚集在森林中央，期待着看到精彩的表演。蓝羽站在起跑线上，心跳加速，但它深吸一口气，闭上眼睛，然后张开翅膀，飞向天空。

起初，蓝羽的飞行并不平稳，但它没有放弃。它努力调整姿态，逐渐掌握了飞行的技巧。当它再次回到地面时，所有的动物都为它鼓掌欢呼。蓝羽不仅赢得了比赛，还证明了自己不仅有美丽的歌声，也有惊人的勇气。

从那以后，蓝羽成为了森林中的英雄，它的故事激励着每一个害怕尝试新事物的生物。而蓝羽和风灵之间的友谊也变得更加深厚，它们一起探索森林的每一个角落，享受着飞行带来的快乐。

这个故事告诉我们，无论我们多么害怕，只要我们勇敢地迈出第一步，就能发现自己的潜力和勇气。<|endoftext|>


In [8]:
model.config.absorb=True
model.save_pretrained("absorb_model")

[2025-02-16 12:08:08,744] [INFO] [real_accelerator.py:219:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlvsym'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlopen'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: undefined reference to `dlclose'
/data/mfx/miniconda3/envs/transmla/bin/../lib/gcc/x86_64-conda-linux-gnu/11.2.0/../../../../x86_64-conda-linux-gnu/bin/ld: /root/miniconda3/envs/transmla/lib/libcufile.so: unde

In [ ]:
model = Qwen2ForCausalLM.from_pretrained("/root/mfx/TransMLA_v2/models/absorb_model", device_map='cuda:1', attn_implementation="eager")
tokenizer = AutoTokenizer.from_pretrained("/root/mfx/huggingface/Qwen/Qwen2.5-7B")
model

Loading checkpoint shards: 100%|██████████| 8/8 [00:12<00:00,  1.60s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2MLAttention(
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (q_proj): Linear(in_features=3584, out_features=14336, bias=True)
          (o_proj): Linear(in_features=14336, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [3]:
output = model.generate(**tokenizer("给我讲一个500字的故事",return_tensors="pt").to("cuda:1"), max_new_tokens=100, do_sample=False)
tokenizer.batch_decode(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


['给我讲一个500字的故事吧。\n好的，这是一个关于勇气和友谊的故事。从前，有一个勇敢的小男孩，他叫做杰克。杰克住在一个小村庄里，他和他的朋友们经常一起玩耍。有一天，他们听说了一个传说，说在森林深处有一个神秘的宝藏，但是只有最勇敢的人才能找到它。杰克和他的朋友们决定一起去寻找这个宝藏。\n\n他们穿过茂密的森林，遇到了许多困难和危险。但是，杰克和他的朋友们互相帮助，一起克服了所有的障碍']